In [1]:
import pandas as pd
from io import StringIO
import os

## Read trace

In [2]:
def read_trace(trace_dir, trace_file, worker_id=None):
    trace_filepath = os.path.join(trace_dir, trace_file)
    
    with open(trace_filepath,"r") as rf:
        new_trace = StringIO()
        line = rf.readline()
        while line:
            line_list = line.split()
            if line_list[0] == "[trace]":
                new_trace.write(line)
            line = rf.readline()
       
    new_trace.seek(0)
    df = pd.read_csv(new_trace, sep=" ", names=["InfoType", "EventType", "EventState", "ID", "Time"])
    del df["InfoType"]
    
    if worker_id is not None:
        df.insert(0, "WorkerID", worker_id)
        
    new_trace.close()
    
    return df

### COORD

In [3]:
trace_dir = "/home/posheng/billy_ws/fl_025/paslab/trace/async2_fl_mp_3rm/resnet_18"
coord_trace_file = "coord.txt"
hpc0_trace_file = "hpc0.txt"
hpc1_trace_file = "hpc1.txt"
hpc2_trace_file = "hpc2.txt"
# hpc3_trace_file = "hpc3_trace.txt"

In [4]:
# coord_df = read_trace(trace_dir, coord_trace_file)
# hpc1_df = read_trace(trace_dir, hpc1_trace_file, "alice")
# hpc2_df = read_trace(trace_dir, hpc2_trace_file, "bob")
# hpc3_df = read_trace(trace_dir, hpc3_trace_file, "charlie")

coord_df = read_trace(trace_dir, coord_trace_file)
w0_df = read_trace(trace_dir, hpc0_trace_file, "alice")
w1_df = read_trace(trace_dir, hpc1_trace_file, "bob")
w2_df = read_trace(trace_dir, hpc2_trace_file, "charlie")

In [5]:
hpc1_df = w0_df
hpc2_df = w1_df
hpc3_df = w2_df

In [6]:
worker_dfs = {
    "alice": hpc1_df,
    "bob": hpc2_df,
    "charlie": hpc3_df
}

## Duration

In [7]:
def get_worker_duration(worker_id, worker_dfs, event_type):
    worker_df = worker_dfs[worker_id]
    worker_event_df = worker_df[worker_df["EventType"].str.contains(event_type, regex=False)]
    worker_event_df = worker_event_df.reset_index()
    
    del worker_event_df['index']
    del worker_event_df['ID']
    
    return worker_event_df

In [8]:
def get_coord_duration(coord_df, event_type):
    coord_event_df = coord_df[coord_df["EventType"].str.contains(event_type, regex=False)]
    coord_event_df = coord_event_df.reset_index()
    
    del coord_event_df['index']
    del coord_event_df['ID']
    
    return coord_event_df

In [9]:
def calculate_duration(event_type, df):
    event_df = df[df["EventType"] == event_type][["EventType", "EventState", "ID", "Time"]]
    event_start_df = event_df[event_df["EventState"] == "start"][["ID", "Time"]]
    event_start_df = event_start_df.reset_index()

    event_end_df = event_df[event_df["EventState"] == "end"][["Time"]]
    event_end_df = event_end_df.reset_index()
    
    event_df_cat = pd.concat([event_start_df, event_end_df], axis=1)
    del event_df_cat['index']
    event_df_cat.columns = ["ID", "StartTime", "EndTime"]
    
    event_df_cat["Duration"] = event_df_cat.apply(lambda x: x["EndTime"] - x["StartTime"], axis=1)
    event_df_cat.insert(0, "EventType", event_type)
    return event_df_cat


In [10]:
GlobalModelSendDuration = calculate_duration("GlobalModelSend", coord_df)

In [11]:
LossFuncSendDuration = calculate_duration("LossFuncSend", coord_df)

In [12]:
ModelConfigSendDuration = calculate_duration("ModelConfigSend", coord_df)

### GlobalModelSendDuration

In [13]:
GlobalModelSendDuration[GlobalModelSendDuration["ID"] == "alice"][1:]['Duration'].mean()

8.621657371520996

In [14]:
GlobalModelSendDuration[GlobalModelSendDuration["ID"] == "bob"][1:]['Duration'].mean()

8.60559024810791

In [15]:
GlobalModelSendDuration[GlobalModelSendDuration["ID"] == "charlie"][1:]['Duration'].mean()

8.568798160552978

### LossFuncSendDuration

In [16]:
LossFuncSendDuration[LossFuncSendDuration["ID"] == "alice"][1:]['Duration'].mean()

0.0016753792762756348

In [17]:
LossFuncSendDuration[LossFuncSendDuration["ID"] == "bob"][1:]['Duration'].mean()

0.0015579938888549804

In [18]:
LossFuncSendDuration[LossFuncSendDuration["ID"] == "charlie"][1:]['Duration'].mean()

0.0016451478004455566

### ModelConfigSendDuration

In [19]:
ModelConfigSendDuration[ModelConfigSendDuration["ID"] == "alice"][1:]['Duration'].mean()

0.0009323477745056153

In [20]:
ModelConfigSendDuration[ModelConfigSendDuration["ID"] == "bob"][1:]['Duration'].mean()

0.0009285807609558105

In [21]:
ModelConfigSendDuration[ModelConfigSendDuration["ID"] == "charlie"][1:]['Duration'].mean()

0.0010310530662536622

## GlobalInformationSend Duration

In [22]:
def get_coord_global_send_duration(coord_df, worker_id):
    coord_event_df = coord_df[coord_df["EventType"].str.contains("GlobalInformationSend", regex=False)]
    coord_event_df = coord_event_df[coord_event_df["ID"] == worker_id]
    coord_event_df = coord_event_df.reset_index()
    
    del coord_event_df['index']
    
    return coord_event_df

In [23]:
hpc1_gs_df = get_coord_global_send_duration(coord_df, "alice")

In [24]:
hpc2_gs_df = get_coord_global_send_duration(coord_df, "bob")

In [25]:
hpc3_gs_df = get_coord_global_send_duration(coord_df, "charlie")

In [26]:
hpc1_gs_df['Time'][1:].mean()

8.664455854892731

In [27]:
hpc2_gs_df['Time'][1:].mean()

8.663996374607086

In [28]:
hpc3_gs_df['Time'][1:].mean()

8.475526988506317

## Send Time

In [29]:
## EventType:
##     * GlobalModelSend
##     * LossFuncSend
##     * ModelConfigSend
def calculate_send_time(coord_df, worker_dfs, event_type):
    event_start_df = coord_df[(coord_df["EventType"] == event_type) & (coord_df["EventState"] == "start")]
    worker_event_send_time_dfs = {}
    
    for worker_id in worker_dfs.keys():
        worker_event_start_df = event_start_df[event_start_df["ID"] == worker_id]
        worker_event_start_df = worker_event_start_df.reset_index()
        del worker_event_start_df['index']
        worker_event_start_df.columns = ["EventType", "EventState", "ID", "StartTime"]
        
        worker_df = worker_dfs[worker_id]
        worker_event_recv_df = worker_df[worker_df["EventType"] == event_type]
        worker_event_recv_df = worker_event_recv_df.reset_index()
        del worker_event_recv_df['index']
        worker_event_recv_df.columns = ["WorkerID", "EventType", "EventState", "ID", "RecvTime"]
        
        worker_recv_time_series = worker_event_recv_df["RecvTime"]
        worker_event_send_time_df = pd.concat([worker_event_start_df, worker_recv_time_series], axis=1)
        worker_event_send_time_df["SendTime"] = worker_event_send_time_df.apply(lambda x: x["RecvTime"] - x["StartTime"], axis=1)
        del worker_event_send_time_df["EventState"]
        
        worker_event_send_time_dfs[worker_id] = worker_event_send_time_df
        
    return worker_event_send_time_dfs

In [30]:
worker_gms_send_time_dfs = calculate_send_time(coord_df, worker_dfs, "GlobalModelSend")

In [31]:
worker_gms_send_time_dfs['alice']['SendTime'].mean()

8.680015881856283

In [32]:
worker_gms_send_time_dfs['alice']

,EventType,ID,StartTime,RecvTime,SendTime
0,GlobalModelSend,alice,1.600982e+09,1.600982e+09,9.057966
1,GlobalModelSend,alice,1.600982e+09,1.600982e+09,8.450757
2,GlobalModelSend,alice,1.600983e+09,1.600983e+09,8.998788
3,GlobalModelSend,alice,1.600983e+09,1.600983e+09,8.528796
4,GlobalModelSend,alice,1.600983e+09,1.600983e+09,8.194410
5,GlobalModelSend,alice,1.600983e+09,1.600983e+09,9.077507
6,GlobalModelSend,alice,1.600983e+09,1.600983e+09,8.533570
7,GlobalModelSend,alice,1.600983e+09,1.600983e+09,8.500657
8,GlobalModelSend,alice,1.600983e+09,1.600983e+09,9.045883
9,GlobalModelSend,alice,1.600983e+09,1.600983e+09,8.692936


In [33]:
worker_gms_send_time_dfs['charlie']['SendTime'].mean()

8.478823775336856

In [34]:
# coord_df = read_trace(trace_dir, coord_trace_file)
# hpc1_df = read_trace(trace_dir, hpc1_trace_file, "alice")
# hpc2_df = read_trace(trace_dir, hpc2_trace_file, "bob")
# hpc3_df = read_trace(trace_dir, hpc3_trace_file, "charlie")

## LocalTraining Duration

In [35]:
hpc1_lt_df = get_worker_duration("alice", worker_dfs, "LocalTraining")

In [36]:
hpc2_lt_df = get_worker_duration("bob", worker_dfs, "LocalTraining")

In [37]:
hpc3_lt_df = get_worker_duration("charlie", worker_dfs, "LocalTraining")

In [38]:
hpc1_lt_df['Time'][1:].mean()

26.757357597351074

In [39]:
hpc2_lt_df['Time'][1:].mean()

18.48927059173584

In [40]:
hpc3_lt_df['Time'][1:].mean()

19.464396154880525

In [41]:
hpc1_lt_df

,WorkerID,EventType,EventState,Time
0,alice,LocalTraining,duration,26.624156
1,alice,LocalTraining,duration,26.722337
2,alice,LocalTraining,duration,26.563231
3,alice,LocalTraining,duration,26.818083
4,alice,LocalTraining,duration,26.732565
5,alice,LocalTraining,duration,26.826841
6,alice,LocalTraining,duration,26.953744
7,alice,LocalTraining,duration,26.687086
8,alice,LocalTraining,duration,26.583683
9,alice,LocalTraining,duration,26.751819


## FitSagg Duration

In [42]:
def get_coord_fitsagg_duration(coord_df, worker_id):
    coord_event_df = coord_df[coord_df["EventType"].str.contains("FitSagg", regex=False)]
    coord_event_df = coord_event_df[coord_event_df["ID"] == worker_id]
    coord_event_df = coord_event_df.reset_index()
    
    del coord_event_df['index']
    
    return coord_event_df

In [43]:
hpc1_fs_df = get_coord_fitsagg_duration(coord_df, "alice")

In [44]:
hpc2_fs_df = get_coord_fitsagg_duration(coord_df, "bob")

In [45]:
hpc3_fs_df = get_coord_fitsagg_duration(coord_df, "charlie")

In [46]:
hpc1_fs_df['Time']

0     84.453076
1     85.615986
2     85.717048
3     91.836997
4     86.045487
5     86.111153
6     98.351408
7     97.899580
8     86.157892
9     86.350277
10    92.211237
11    92.692703
12    86.314981
13    97.601875
14    85.350809
15    97.979316
16    97.995654
17    86.370761
18    83.190006
19    85.936923
20    86.306107
Name: Time, dtype: float64

In [47]:
hpc1_fs_df['Time'][1:].mean()

89.80181002616882

In [48]:
hpc2_fs_df['Time'][1:].mean()

94.73016160726547

In [49]:
hpc3_fs_df['Time'][1:].mean()

93.92386130094528

## EncryptMultiply Duration

In [50]:
hpc1_em_df = get_worker_duration("alice", worker_dfs, "EncryptMultiply")

In [51]:
hpc2_em_df = get_worker_duration("bob", worker_dfs, "EncryptMultiply")

In [52]:
hpc3_em_df = get_worker_duration("charlie", worker_dfs, "EncryptMultiply")

In [53]:
hpc1_em_df['Time'][1:].mean()

54.00186846256256

In [54]:
hpc2_em_df['Time'][1:].mean()

66.99678223133087

In [55]:
hpc3_em_df['Time'][1:].mean()

62.7054208278656

In [56]:
hpc1_em_df

,WorkerID,EventType,EventState,Time
0,alice,EncryptMultiply,duration,48.715663
1,alice,EncryptMultiply,duration,50.059990
2,alice,EncryptMultiply,duration,50.112348
3,alice,EncryptMultiply,duration,56.116772
4,alice,EncryptMultiply,duration,50.318476
5,alice,EncryptMultiply,duration,50.170261
6,alice,EncryptMultiply,duration,62.448335
7,alice,EncryptMultiply,duration,61.933868
8,alice,EncryptMultiply,duration,50.497008
9,alice,EncryptMultiply,duration,50.482827


In [57]:
hpc3_em_df

,WorkerID,EventType,EventState,Time
0,charlie,EncryptMultiply,duration,61.433725
1,charlie,EncryptMultiply,duration,55.777690
2,charlie,EncryptMultiply,duration,63.421010
3,charlie,EncryptMultiply,duration,68.569799
4,charlie,EncryptMultiply,duration,57.262148
5,charlie,EncryptMultiply,duration,57.845465
6,charlie,EncryptMultiply,duration,69.089931
7,charlie,EncryptMultiply,duration,74.851400
8,charlie,EncryptMultiply,duration,57.915925
9,charlie,EncryptMultiply,duration,58.021731


## EncryptParameter Duration

In [58]:
def get_encrypt_parameter_duration(worker_id, worker_dfs):
    worker_df = worker_dfs[worker_id]
    worker_ep_df = worker_df[worker_df["EventType"].str.contains('EncryptParameter', regex=False)]
    worker_ep_df = worker_ep_df.reset_index()
    
    del worker_ep_df["index"]
    del worker_ep_df["ID"]
    
    return worker_ep_df
    

In [59]:
# hpc1_ep_df = get_encrypt_parameter_duration("alice", worker_dfs)
hpc1_ep_df = get_worker_duration("alice", worker_dfs, "EncryptParameter")
hpc2_ep_df = get_worker_duration("bob", worker_dfs, "EncryptParameter")
hpc3_ep_df = get_worker_duration("charlie", worker_dfs, "EncryptParameter")

In [60]:
hpc1_ep_df[0:10]

,WorkerID,EventType,EventState,Time
0,alice,EncryptParameter0,duration,0.008006
1,alice,EncryptParameter1,duration,0.004122
2,alice,EncryptParameter2,duration,0.135807
3,alice,EncryptParameter3,duration,0.042323
4,alice,EncryptParameter4,duration,20.101438
5,alice,EncryptParameter5,duration,0.049372
6,alice,EncryptParameter6,duration,20.748300
7,alice,EncryptParameter7,duration,0.470586
8,alice,EncryptParameter8,duration,0.029997
9,alice,EncryptParameter9,duration,0.005698


In [61]:
mean_values = []
for i in range(8):
    mean_value = hpc1_ep_df[hpc1_ep_df["EventType"] == "EncryptParameter"+str(i)]["Time"].mean()
    mean_values.append(mean_value)
    print(str(i)+":", mean_value)

0: 0.008286589667910621
1: 0.004443316232590448
2: 0.11157965660095215
3: 0.08461517379397437
4: 28.520718915121897
5: 0.2607398260207403
6: 16.35787110101609
7: 0.48934257598150344


In [62]:
sum(mean_values)

45.837597154435656

In [63]:
for i in range(8):
    mean_value = hpc2_ep_df[hpc1_ep_df["EventType"] == "EncryptParameter"+str(i)]["Time"].mean()
    print(str(i)+":", mean_value)

0: 8.331863948277064
1: 0.004645381655011859
2: 0.13638104711260116
3: 1.0870293435596285
4: 33.396468003590904
5: 0.6185829298836845
6: 16.398947988237655
7: 0.006813072022937593


In [64]:
for i in range(8):
    mean_value = hpc3_ep_df[hpc1_ep_df["EventType"] == "EncryptParameter"+str(i)]["Time"].mean()
    print(str(i)+":", mean_value)

0: 7.37497479575021
1: 0.004630304518200103
2: 0.10904356411525182
3: 0.08928410212198894
4: 26.07693376995268
5: 0.1528927939278739
6: 17.312533106122697
7: 0.011078516642252604


In [65]:
hpc1_ep_df["Time"][0:8]

0     0.008006
1     0.004122
2     0.135807
3     0.042323
4    20.101438
5     0.049372
6    20.748300
7     0.470586
Name: Time, dtype: float64

In [66]:
hpc1_ep_df["Time"].mean()

4.670967030525207

In [67]:
hpc2_ep_df["Time"].mean()

6.001383661088489

In [68]:
hpc3_ep_df["Time"].mean()

5.1643153735569545

## MultiplyParameter Duration

In [69]:
def get_multiply_parameter_duration(worker_id, worker_dfs):
    worker_df = worker_dfs[worker_id]
    worker_ep_df = worker_df[worker_df["EventType"].str.contains('MultiplyParameter', regex=False)]
    worker_ep_df = worker_ep_df.reset_index()
    
    del worker_ep_df["index"]
    del worker_ep_df["ID"]
    
    return worker_ep_df

In [70]:
# hpc1_mp_df = get_multiply_parameter_duration("alice", worker_dfs)
hpc1_mp_df = get_worker_duration("alice", worker_dfs, "MultiplyParameter")
hpc2_mp_df = get_worker_duration("bob", worker_dfs, "MultiplyParameter")
hpc3_mp_df = get_worker_duration("charlie", worker_dfs, "MultiplyParameter")

In [71]:
hpc1_mp_df[:8]

,WorkerID,EventType,EventState,Time
0,alice,MultiplyParameter0,duration,0.005097
1,alice,MultiplyParameter1,duration,0.004630
2,alice,MultiplyParameter2,duration,0.003384
3,alice,MultiplyParameter3,duration,0.002941
4,alice,MultiplyParameter4,duration,1.342498
5,alice,MultiplyParameter5,duration,0.006139
6,alice,MultiplyParameter6,duration,5.683403
7,alice,MultiplyParameter7,duration,0.004724


In [72]:
hpc1_mp_df[:8]["Time"].sum()

7.05281662940979

## RetrieveTime duration

In [73]:
def get_coord_retrieve_duration(coord_df, worker_id):
    coord_event_df = coord_df[coord_df["EventType"].str.contains("RetrieveTime", regex=False)]
    coord_event_df = coord_event_df[coord_event_df["ID"] == worker_id]
    coord_event_df = coord_event_df.reset_index()
    
    del coord_event_df['index']
    
    return coord_event_df

In [74]:
hpc1_ret_df = get_coord_retrieve_duration(coord_df, "alice")

In [75]:
hpc2_ret_df = get_coord_retrieve_duration(coord_df, "bob")

In [76]:
hpc3_ret_df = get_coord_retrieve_duration(coord_df, "charlie")

In [77]:
hpc1_ret_df["Time"][1:].mean()

0.021232306957244873

In [78]:
hpc2_ret_df["Time"][1:].mean()

0.019333517551422118

In [79]:
hpc3_ret_df["Time"][1:].mean()

1.0614069104194641

## ShareSend Duration

In [80]:
def calculate_share_send_duration(worker_id, worker_dfs, nr_parameters):
    worker_df = worker_dfs[worker_id]
    worker_send_share_df = worker_df[worker_df["EventType"].str.contains('SendShare', regex=False)]
    
    worker_send_share_start_df = worker_send_share_df[worker_send_share_df["EventState"] == 'start']
    worker_send_share_start_df = worker_send_share_start_df.reset_index()
    worker_send_share_start_df = worker_send_share_start_df[["WorkerID", "EventType", "ID", "Time"]]
    worker_send_share_start_df.columns = ["WorkerID", "EventType", "ID", "StartTime"]
    
    worker_send_share_end_df = worker_send_share_df[worker_send_share_df["EventState"] == 'end']
    worker_send_share_end_df = worker_send_share_end_df.reset_index()
    
    send_share_df_cat = pd.concat([worker_send_share_start_df, worker_send_share_end_df["Time"]], axis=1)
    send_share_df_cat.columns = ["WorkerID", "EventType", "ID", "StartTime", "EndTime"]
    send_share_df_cat["Duration"] = send_share_df_cat.apply(lambda x: x["EndTime"] - x["StartTime"], axis=1)
    
    parameter_index = []
    nr_worker = len(worker_dfs)
    p_index = -1

    for i in range(len(send_share_df_cat)):
        tmp_index = (i%nr_worker)
        if tmp_index == 0:
            p_index += 1
            p_index = p_index % nr_parameters
        parameter_index.append(p_index)
        
    send_share_df_cat.insert(2, "p_idx", parameter_index)
    obj_id_list = send_share_df_cat.apply(lambda x: x["EventType"].split("_")[1], axis=1)
    send_share_df_cat.insert(2, "obj_id", obj_id_list)
    
    del send_share_df_cat["EventType"]
    send_share_df_cat.insert(1, "EventType", "SendShare")
    
    return send_share_df_cat


In [81]:
alice_send_share_df_cat = calculate_share_send_duration("alice", worker_dfs, 8)
bob_send_share_df_cat = calculate_share_send_duration("bob", worker_dfs, 8)
charlie_send_share_df_cat = calculate_share_send_duration("charlie", worker_dfs, 8)

In [82]:
alice_send_share_df_cat

,WorkerID,EventType,obj_id,p_idx,ID,StartTime,EndTime,Duration
0,alice,SendShare,81726484208,0,alice,1.600989e+09,1.600989e+09,0.001325
1,alice,SendShare,7021502281,0,bob,1.600989e+09,1.600989e+09,0.004389
2,alice,SendShare,21102619797,0,charlie,1.600989e+09,1.600989e+09,0.003375
3,alice,SendShare,47368133411,1,alice,1.600989e+09,1.600989e+09,0.000543
4,alice,SendShare,42225915484,1,bob,1.600989e+09,1.600989e+09,0.001389
...,...,...,...,...,...,...,...,...
625,alice,SendShare,64725182585,0,bob,1.600993e+09,1.600993e+09,0.010800
626,alice,SendShare,47839148087,0,charlie,1.600993e+09,1.600993e+09,0.012750
627,alice,SendShare,30827963345,1,alice,1.600993e+09,1.600993e+09,0.000837
628,alice,SendShare,35995450102,1,bob,1.600993e+09,1.600993e+09,0.012508


In [83]:
bob_send_share_df_cat

,WorkerID,EventType,obj_id,p_idx,ID,StartTime,EndTime,Duration
0,bob,SendShare,86966281982,0,alice,1.600989e+09,1.600989e+09,14.264601
1,bob,SendShare,92796097644,0,bob,1.600989e+09,1.600989e+09,0.001018
2,bob,SendShare,47884369046,0,charlie,1.600989e+09,1.600989e+09,0.004379
3,bob,SendShare,15245692783,1,alice,1.600989e+09,1.600989e+09,0.001692
4,bob,SendShare,35723609579,1,bob,1.600989e+09,1.600989e+09,0.000601
...,...,...,...,...,...,...,...,...
625,bob,SendShare,74539441363,0,bob,1.600993e+09,1.600993e+09,0.000688
626,bob,SendShare,59367873517,0,charlie,1.600993e+09,1.600993e+09,0.014472
627,bob,SendShare,10555305712,1,alice,1.600993e+09,1.600993e+09,0.001903
628,bob,SendShare,99926598382,1,bob,1.600993e+09,1.600993e+09,0.000851


In [84]:
charlie_send_share_df_cat

,WorkerID,EventType,obj_id,p_idx,ID,StartTime,EndTime,Duration
0,charlie,SendShare,58572881221,0,alice,1.600989e+09,1.600989e+09,12.746826
1,charlie,SendShare,19472077688,0,bob,1.600989e+09,1.600989e+09,0.004589
2,charlie,SendShare,10945524671,0,charlie,1.600989e+09,1.600989e+09,0.001003
3,charlie,SendShare,6105564876,1,alice,1.600989e+09,1.600989e+09,0.001767
4,charlie,SendShare,39157060471,1,bob,1.600989e+09,1.600989e+09,0.001618
...,...,...,...,...,...,...,...,...
625,charlie,SendShare,27670397130,0,bob,1.600993e+09,1.600993e+09,0.014488
626,charlie,SendShare,49790727356,0,charlie,1.600993e+09,1.600993e+09,0.000716
627,charlie,SendShare,5909691836,1,alice,1.600993e+09,1.600993e+09,0.001620
628,charlie,SendShare,94685771184,1,bob,1.600993e+09,1.600993e+09,10.437300


## ShareSend Send Time

In [85]:
def calculate_share_send_time(sender_id, receiver_id, worker_dfs, nr_parameter):
    sender_df = worker_dfs[sender_id]
    receiver_df = worker_dfs[receiver_id]
    
    sender_send_share_df = sender_df[sender_df["EventType"].str.contains('SendShare', regex=False)]
    receiver_get_share_df = receiver_df[receiver_df["EventType"].str.contains('GetShare', regex=False)]
    
    send_share_start_df = sender_send_share_df[(sender_send_share_df["ID"] == receiver_id) & (sender_send_share_df["EventState"] == 'start')]
    get_share_df = receiver_get_share_df[receiver_get_share_df["ID"] == sender_id]
    send_share_start_df = send_share_start_df.reset_index()
    get_share_df = get_share_df.reset_index()
    
    del send_share_start_df['index']
    del send_share_start_df['EventState']
    
    send_time_df = pd.concat([send_share_start_df, get_share_df["Time"]], axis=1)
    send_time_df.columns = ["WorkerID", "EventType", "ID", "StartTime", "RecvTime"]
    send_time_df["SendTime"] = send_time_df.apply(lambda x: x["RecvTime"] - x["StartTime"], axis=1)
    
    nr_parameter = 8
    parameter_index = []

    for i in range(len(send_time_df)):
        tmp_index = (i%nr_parameter)
        parameter_index.append(tmp_index)
        
    send_time_df.insert(2, "p_idx", parameter_index)
    
    obj_id_list = send_time_df.apply(lambda x: x["EventType"].split("_")[1], axis=1)
    send_time_df.insert(2, "obj_id", obj_id_list)
    
    del send_time_df["EventType"]
    send_time_df.insert(1, "EventType", "SendShare")
    
    return send_time_df

In [86]:
send_time_df = calculate_share_send_time("alice", "bob", worker_dfs, 8)

In [87]:
send_time_df

,WorkerID,EventType,obj_id,p_idx,ID,StartTime,RecvTime,SendTime
0,alice,SendShare,7021502281,0,bob,1.600989e+09,1.600989e+09,0.002271
1,alice,SendShare,42225915484,1,bob,1.600989e+09,1.600989e+09,0.000004
2,alice,SendShare,95246673600,2,bob,1.600989e+09,1.600989e+09,0.046076
3,alice,SendShare,1788088392,3,bob,1.600989e+09,1.600989e+09,0.000487
4,alice,SendShare,9764687758,4,bob,1.600989e+09,1.600989e+09,19.329998
...,...,...,...,...,...,...,...,...
205,alice,SendShare,44234087574,5,bob,1.600993e+09,1.600993e+09,0.029216
206,alice,SendShare,94309519198,6,bob,1.600993e+09,1.600993e+09,11.570052
207,alice,SendShare,65707164086,7,bob,1.600993e+09,1.600993e+09,1.043782
208,alice,SendShare,64725182585,0,bob,1.600993e+09,1.600993e+09,0.009126


## AllWorkersTrainingTime Duration

In [88]:
coord_awtt_df = get_coord_duration(coord_df, "AllWorkersTrainingTime")

In [89]:
coord_awtt_df['Time'][1:].mean()

191.40200626850128

In [90]:
coord_awtt_df

,EventType,EventState,Time
0,AllWorkersTrainingTime,duration,184.972239
1,AllWorkersTrainingTime,duration,189.774938
2,AllWorkersTrainingTime,duration,203.889259
3,AllWorkersTrainingTime,duration,178.882474
4,AllWorkersTrainingTime,duration,175.176354
5,AllWorkersTrainingTime,duration,187.177509
6,AllWorkersTrainingTime,duration,174.767891
7,AllWorkersTrainingTime,duration,185.905089
8,AllWorkersTrainingTime,duration,187.546048
9,AllWorkersTrainingTime,duration,207.651450


## AggregationTime

In [91]:
coord_aggt_df = get_coord_duration(coord_df, "AggregationTime")

In [92]:
coord_aggt_df["Time"][1:].mean()

0.3642228603363037

In [93]:
coord_aggt_df

,EventType,EventState,Time
0,AggregationTime,duration,0.348947
1,AggregationTime,duration,0.372648
2,AggregationTime,duration,0.366201
3,AggregationTime,duration,0.359317
4,AggregationTime,duration,0.357176
5,AggregationTime,duration,0.364825
6,AggregationTime,duration,0.365436
7,AggregationTime,duration,0.366344
8,AggregationTime,duration,0.364848
9,AggregationTime,duration,0.376091


## DecryptionTime

In [94]:
coord_decrypt_df = get_coord_duration(coord_df, "DecryptionTime")

In [95]:
coord_decrypt_df['Time'][1:].mean()

10.786752891540527

In [96]:
coord_decrypt_df

,EventType,EventState,Time
0,DecryptionTime,duration,10.622688
1,DecryptionTime,duration,10.801486
2,DecryptionTime,duration,10.770778
3,DecryptionTime,duration,10.795599
4,DecryptionTime,duration,10.886293
5,DecryptionTime,duration,10.880650
6,DecryptionTime,duration,10.658896
7,DecryptionTime,duration,10.798456
8,DecryptionTime,duration,10.785601
9,DecryptionTime,duration,10.822168


## RoundTime

In [97]:
coord_round_time_df = get_coord_duration(coord_df, "RoundTime")

In [98]:
coord_round_time_df["Time"].mean()

202.36223678361802

In [153]:
coord_round_time_df

,EventType,EventState,Time
0,RoundTime,duration,203.230953
1,RoundTime,duration,200.259282
2,RoundTime,duration,185.037489
3,RoundTime,duration,199.478490
4,RoundTime,duration,174.921678
5,RoundTime,duration,198.649328
6,RoundTime,duration,175.031498
7,RoundTime,duration,184.907219
8,RoundTime,duration,174.366345
9,RoundTime,duration,184.226114
